In [ ]:
import numpy as np
import pandas as pd
import spacy
import pandas as pd

In [ ]:
# Loading NLP
nlp = spacy.load('en_core_web_lg')

#Loading the train data
train_data = pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
#Loading the vectors
with nlp.disable_pipes():
    vectors = np.array([nlp(text.text).vector for idx, text in train_data.iterrows()])

## Center the vectors
# Calculate the mean for the vectors
vec_mean = vectors.mean(axis=0)
# Subtract the mean from the vectors
centered = vectors-vec_mean

In [ ]:
#Training the model
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(centered , train_data.target, 
                                                    test_size=0.1, random_state=1)

#Lets perform scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_val_scaled = scaler.transform(X_val.astype(np.float32))

# Create the SVC model
model = SVC(gamma='scale')

# Fit the model
model.fit(X_train_scaled,y_train)

# Check model accuracy
print(f'Model test accuracy: {model.score(X_val_scaled, y_val)*100:.3f}%')

In [ ]:
#Reading the test data
test_data = pd.read_csv('../input/nlp-getting-started/test.csv')

#Loading the test vectors
with nlp.disable_pipes():
    test_vectors = np.array([nlp(text.text).vector for idx, text in test_data.iterrows()])

## Center the test vectors
# Calculate the mean for the vectors
vec_mean_test = test_vectors.mean(axis=0)
# Subtract the mean from the vectors
centered_test = test_vectors-vec_mean_test
    
#Perform scaling
X_test_scaled = scaler.fit_transform(centered_test.astype(np.float32))

#Predict
predictions=model.predict(X_test_scaled)

In [ ]:
#Prepare it in the format for submission
predictions_df=pd.DataFrame(predictions,index=test_data.index,columns=['target'])
predictions_df=test_data.join(predictions_df.target)
predictions_df=predictions_df.drop(columns=['keyword','location','text'])
predictions_df.to_csv('submission.csv', index=False)